In [ ]:
pip install requests pandas matplotlib


#main.py - Main Application

In [ ]:
import requests
import sqlite3
import time
import pandas as pd
import smtplib
import matplotlib.pyplot as plt
from datetime import datetime

# OpenWeatherMap API Key and Base URL
API_KEY = 'your_api_key_here'
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather'
CITIES = ['Delhi', 'Mumbai', 'Chennai', 'Bangalore', 'Kolkata', 'Hyderabad']
INTERVAL = 300  # 5 minutes

# Database setup
def init_db():
    conn = sqlite3.connect('weather.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS weather_data
                 (city TEXT, temp REAL, feels_like REAL, main TEXT, dt INTEGER)''')
    c.execute('''CREATE TABLE IF NOT EXISTS daily_summary
                 (city TEXT, date TEXT, avg_temp REAL, max_temp REAL, min_temp REAL, dominant_condition TEXT)''')
    conn.commit()
    return conn

# Fetch weather data from OpenWeather API
def fetch_weather(city):
    params = {'q': city, 'appid': API_KEY}
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    return {
        'city': city,
        'temp': kelvin_to_celsius(data['main']['temp']),
        'feels_like': kelvin_to_celsius(data['main']['feels_like']),
        'main': data['weather'][0]['main'],
        'dt': data['dt']
    }

# Convert temperature from Kelvin to Celsius
def kelvin_to_celsius(temp_kelvin):
    return temp_kelvin - 273.15

# Insert data into the weather_data table
def insert_weather_data(conn, data):
    c = conn.cursor()
    c.execute("INSERT INTO weather_data (city, temp, feels_like, main, dt) VALUES (?, ?, ?, ?, ?)", 
              (data['city'], data['temp'], data['feels_like'], data['main'], data['dt']))
    conn.commit()

# Calculate daily summaries (rollups)
def calculate_daily_summary(conn):
    c = conn.cursor()
    today = datetime.now().strftime('%Y-%m-%d')
    for city in CITIES:
        c.execute('''SELECT temp, main FROM weather_data WHERE city=? AND dt >= strftime('%s', 'now', 'start of day')''', (city,))
        rows = c.fetchall()
        if rows:
            temps = [r[0] for r in rows]
            weather_conditions = [r[1] for r in rows]
            avg_temp = sum(temps) / len(temps)
            max_temp = max(temps)
            min_temp = min(temps)
            dominant_condition = max(set(weather_conditions), key=weather_conditions.count)  # Most frequent condition
            c.execute('''INSERT INTO daily_summary (city, date, avg_temp, max_temp, min_temp, dominant_condition)
                         VALUES (?, ?, ?, ?, ?, ?)''', (city, today, avg_temp, max_temp, min_temp, dominant_condition))
    conn.commit()

# Alert based on threshold breaches
def check_threshold(conn, threshold_temp=35):
    c = conn.cursor()
    for city in CITIES:
        c.execute('''SELECT temp FROM weather_data WHERE city=? ORDER BY dt DESC LIMIT 2''', (city,))
        temps = c.fetchall()
        if len(temps) == 2 and all(t[0] > threshold_temp for t in temps):
            print(f'Alert: {city} has exceeded {threshold_temp}°C for two consecutive updates!')

# Visualization
def visualize_data(conn):
    c = conn.cursor()
    today = datetime.now().strftime('%Y-%m-%d')
    for city in CITIES:
        c.execute('''SELECT avg_temp, max_temp, min_temp FROM daily_summary WHERE city=? AND date=?''', (city, today))
        summary = c.fetchone()
        if summary:
            plt.bar(['Avg Temp', 'Max Temp', 'Min Temp'], summary)
            plt.title(f'{city} - Weather Summary for {today}')
            plt.ylabel('Temperature (°C)')
            plt.show()

# Main loop to continuously fetch and process weather data
def main():
    conn = init_db()
    while True:
        for city in CITIES:
            weather_data = fetch_weather(city)
            insert_weather_data(conn, weather_data)
        calculate_daily_summary(conn)
        check_threshold(conn)
        visualize_data(conn)
        time.sleep(INTERVAL)

if __name__ == '__main__':
    main()


In [ ]:

FROM python:3.9

WORKDIR /app

COPY . /app

RUN pip install -r requirements.txt

CMD ["python", "main.py"]
